<a href="https://colab.research.google.com/github/Anchayadav/AutoTS/blob/main/DataVision_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install magenta

In [ ]:
###Implemet the code Audio

In [ ]:
###Implement  focus on genre-based music  generation: no thematic analysis of input tex is required
import magenta.music as mm
from magenta.models.music_vae import TrainedModel
from magenta.models.music_vae import configs

def generate_genre_based_music(genre, output_path='generated_music.mid'):

    model_config = configs.CONFIG_MAP['cat-mel_2bar_big']
    model = TrainedModel(model_config, batch_size=4, checkpoint_dir_or_path='cat-mel_2bar_big.tar')


    genre_conditions = {
        'pop': [1, 0, 0, 0],
        'rock': [0, 1, 0, 0],
        'jazz': [0, 0, 1, 0],
        'classical': [0, 0, 0, 1],
    }

    condition = genre_conditions.get(genre, [0, 0, 0, 0])


    generated_sequence = model.sample(n=1, length=128, condition=condition)

    mm.sequence_proto_to_midi_file(generated_sequence[0], output_path)

if __name__ == "__main__":
    selected_genre = 'rock'
    generate_genre_based_music(selected_genre)

In [ ]:

####implement the music recommendation of the song
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split


data = [
    ('user1', 'song1', 5),
    ('user1', 'song2', 4),
    ('user2', 'song1', 3),

]


reader = Reader(rating_scale=(1, 5))

# Load the dataset
dataset = Dataset.load_from_df(data, reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(dataset, test_size=0.2)

# Use the k-NN algorithm for collaborative filtering
sim_options = {
    'name': 'cosine',
    'user_based': False  ,
}

model = KNNBasic(sim_options=sim_options)

# Train the model on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)


user_id = 'user1'
song_id = 'song1'


top_n = model.get_neighbors(model.trainset.to_inner_uid(user_id), k=5)

print(f"Top 5 song recommendations for {user_id}:")
for song_inner_id in top_n:
    song_raw_id = model.trainset.to_raw_iid(song_inner_id)
    print(f"Song ID: {song_raw_id}")


predicted_rating = model.predict(user_id, song_id).est
print(f"Predicted rating for {user_id} and {song_id}: {predicted_rating}")

In [ ]:
### 500 song generated

import magenta.music as mm
from magenta.models.music_vae import TrainedModel
from magenta.models.music_vae import configs

def fine_tune_musicvae(client_songs, output_dir='fine_tuned_model'):

    model_config = configs.CONFIG_MAP['cat-mel_2bar_big']
    model = TrainedModel(model_config, batch_size=4, checkpoint_dir_or_path='cat-mel_2bar_big.tar')


    note_sequences = [mm.midi_file_to_note_sequence(song_path) for song_path in client_songs]


    model.train(note_sequences, output_dir)


client_songs = ['song1.mid', 'song2.mid', ...]  s
fine_tune_musicvae(client_songs)

In [ ]:
####Implement download option  at up to 320kbps quality

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='your_client_id', client_secret='your_client_secret'))

def download_track(track_id, quality='320k'):
    track_info = sp.track(track_id)
    track_name = track_info['name']
    artist_name = track_info['artists'][0]['name']


    available_tracks = sp.track_audio_features(track_id)
    selected_track = None

    for track in available_tracks:
        if track['bitrate'] == quality:
            selected_track = track
            break

    if selected_track:
        audio_url = selected_track['preview_url']

        print(f"Downloading '{track_name}' by {artist_name} at {quality} quality: {audio_url}")
    else:
        print(f"No {quality} quality track available for '{track_name}' by {artist_name}")

track_id = 'your_track_id'
download_track(track_id)


In [ ]:
###IMPlement the Code Video Assignment Code

In [ ]:
##Define the preferredd resolution for the generated video (up to 4k) and ensure a highly-quaity output

import imageio
import numpy as np

def generate_high_quality_video(output_path, resolution=(1920, 1080), duration=10, fps=30):

    num_frames = duration * fps
    frames = [np.random.randint(0, 255, size=(resolution[1], resolution[0], 3), dtype=np.uint8) for _ in range(num_frames)]

    writer = imageio.get_writer(output_path, fps=fps, quality=9)
    for frame in frames:
        writer.append_data(frame)
    writer.close()


output_video_path = 'output_video.mp4'
generate_high_quality_video(output_video_path, resolution=(3840, 2160), duration=10, fps=30)

In [ ]:
 ###Develop technique for achiving fluid motion and dynamic transition in generated video

import imageio
import numpy as np

def generate_smooth_transition_video(output_path, resolution=(1920, 1080), duration=10, fps=30):
    num_frames = duration * fps
    frames = []

    for t in range(num_frames):
        frame = generate_frame(t, num_frames, resolution)
        frames.append(frame)


    writer = imageio.get_writer(output_path, fps=fps)
    for frame in frames:
        writer.append_data(frame)
    writer.close()

def generate_frame(t, num_frames, resolution):

    color = hsv_to_rgb((t / num_frames, 1.0, 1.0))
    frame = np.zeros((resolution[1], resolution[0], 3), dtype=np.uint8)
    frame[:, :] = (color * 255).astype(np.uint8)
    return frame

def hsv_to_rgb(hsv):
    h, s, v = hsv
    if s == 0.0:
        return v, v, v
    i = int(h * 6.0)
    f = (h * 6.0) - i
    p = v * (1.0 - s)
    q = v * (1.0 - s * f)
    t = v * (1.0 - s * (1.0 - f))
    i = i % 6
    if i == 0: return v, t, p
    if i == 1: return q, v, p
    if i == 2: return p, v, t
    if i == 3: return p, q, v
    if i == 4: return t, p, v
    if i == 5: return v, p, q


output_video_path = 'smooth_transition_video.mp4'
generate_smooth_transition_video(output_video_path, resolution=(1920, 1080), duration=10, fps=30)

In [ ]:
##### Implement a user-friends System to text input considering  manual input and file upload option




def get_text_input():
    print("Choose an option:")
    print("1. Manual Input")
    print("2. File Upload")

    option = input("Enter the option number: ")

    if option == '1':
        return manual_input()
    elif option == '2':
        return file_upload()
    else:
        print("Invalid option. Please choose 1 or 2.")
        return get_text_input()

def manual_input():
    return input("Enter your text: ")

def file_upload():
    file_path = input("Enter the path to the file: ")

    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print("File not found. Please enter a valid file path.")
        return file_upload()
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None

def main():
    text_input = get_text_input()

    if text_input:
        print("\nYour input:")
        print(text_input)

if __name__ == "__main__":
    main()

In [ ]:

##Provide user with option to customize visual element such as front color. and background
import colorama
from colorama import Fore, Style

def get_text_input():
    print("Choose an option:")
    print("1. Manual Input")
    print("2. File Upload")

    option = input("Enter the option number: ")

    if option == '1':
        return manual_input()
    elif option == '2':
        return file_upload()
    else:
        print("Invalid option. Please choose 1 or 2.")
        return get_text_input()

def manual_input():
    text_input = input("Enter your text: ")


    font_color = input("Enter font color (e.g., red, green, blue): ")
    background_color = input("Enter background color (e.g., white, yellow, magenta): ")

    return format_text(text_input, font_color, background_color)

def file_upload():
    file_path = input("Enter the path to the file: ")

    try:
        with open(file_path, 'r') as file:
            text_input = file.read()


        font_color = input("Enter font color (e.g., red, green, blue): ")
        background_color = input("Enter background color (e.g., white, yellow, magenta): ")

        return format_text(text_input, font_color, background_color)
    except FileNotFoundError:
        print("File not found. Please enter a valid file path.")
        return file_upload()
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None

def format_text(text, font_color, background_color):
    colorama.init(autoreset=True)


    color_code_map = {
        'black': Fore.BLACK,
        'red': Fore.RED,
        'green': Fore.GREEN,
        'yellow': Fore.YELLOW,
        'blue': Fore.BLUE,
        'magenta': Fore.MAGENTA,
        'cyan': Fore.CYAN,
        'white': Fore.WHITE
    }

    font_color_code = color_code_map.get(font_color.lower(), Fore.RESET)
    background_color_code = color_code_map.get(background_color.lower(), Fore.RESET)

    formatted_text = f"{background_color_code}{font_color_code}{text}{Fore.RESET}"

    return formatted_text

def main():
    text_input = get_text_input()

    if text_input:
        print("\nYour formatted input:")
        print(text_input)

if __name__ == "__main__":
    main()

In [ ]:
######   Detial  if audio integration. such as background music or voiceover should be include in the generated video
###


from moviepy.editor import VideoClip, TextClip, AudioFileClip
from moviepy.audio.io.ffmpeg_audiowriter import FFMPEG_AudioWriter
from moviepy.video.io.VideoFileClip import VideoFileClip

def generate_video_with_audio(output_path, text, background_music_path, voiceover_path):
    # Create a text clip
    text_clip = TextClip(text, fontsize=30, color='white', bg_color='black', size=(640, 480))

    background_music = AudioFileClip(background_music_path)


    voiceover = AudioFileClip(voiceover_path)

    video_duration = max(background_music.duration, voiceover.duration)
    text_clip = text_clip.set_duration(video_duration)


    video_clip = text_clip.set_audio(background_music.subclip(0, text_clip.duration))
    video_clip = video_clip.set_audio(video_clip.audio.set_audio(voiceover.subclip(0, text_clip.duration)))


    video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")


output_video_path = 'output_video_with_audio.mp4'
text_to_display = 'Hello, this is a generated video!'
background_music_file_path = 'background_music.mp3'
voiceover_file_path = 'voiceover.mp3'

generate_video_with_audio(output_video_path, text_to_display, background_music_file_path, voiceover_file_path)

In [ ]:
 #### Determine addition output format or file type to support along side video

from moviepy.editor import VideoClip, TextClip, AudioFileClip

def generate_video_with_audio(output_path, text, background_music_path, voiceover_path, output_format='mp4'):

    text_clip = TextClip(text, fontsize=30, color='white', bg_color='black', size=(640, 480))


    background_music = AudioFileClip(background_music_path)


    voiceover = AudioFileClip(voiceover_path)


    video_duration = max(background_music.duration, voiceover.duration)
    text_clip = text_clip.set_duration(video_duration)

    video_clip = text_clip.set_audio(background_music.subclip(0, text_clip.duration))
    video_clip = video_clip.set_audio(video_clip.audio.set_audio(voiceover.subclip(0, text_clip.duration)))


    if output_format.lower() == 'mp4':
        video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
    elif output_format.lower() == 'avi':
        video_clip.write_videofile(output_path, codec="rawvideo", audio_codec="pcm_s16le")
    else:
        print(f"Unsupported output format: {output_format}")

output_video_path_mp4 = 'output_video_with_audio.mp4'
output_video_path_avi = 'output_video_with_audio.avi'
text_to_display = 'Hello, this is a generated video!'
background_music_file_path = 'background_music.mp3'
voiceover_file_path = 'voiceover.mp3'

generate_video_with_audio(output_video_path_mp4, text_to_display, background_music_file_path, voiceover_file_path, output_format='mp4')
generate_video_with_audio(output_video_path_avi, text_to_display, background_music_file_path, voiceover_file_path, output_format='avi')